# Saving Keras Models in Tensorflow

Since we are using a Tensorflow backend for Keras, we can export a Tensorflow model and upload to Google Cloud ML Service

This will allow us to serve as many users as we need

To do this, we need to add some code to our previous model building process

Once process is finished, new folder (exported_mod) should contain variables and a model in .pb format

In [1]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf

from keras.models import Sequential
from keras.layers import *

C:\Users\casey\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
tf.VERSION

'1.14.0'

In [3]:
training_df = pd.read_csv("sales_training_data_scaled.csv")

X = training_df.drop('total_earnings', axis = 1)
y = training_df['total_earnings']

In [4]:
# rerun model as we did last chapter

model = Sequential()

model.add(Dense(50, input_dim = 9, activation='relu', name = 'layer_1'))
model.add(Dense(100, activation='relu', name = 'layer_2'))
model.add(Dense(50, activation='relu', name = 'layer_3'))
model.add(Dense(1, activation='linear', name = 'output_layer'))

model.compile(loss = 'mse', optimizer = 'adam')

logger = keras.callbacks.TensorBoard(log_dir='logs', write_graph=True)

model.fit(X, y, epochs = 50, shuffle = True, verbose = 2, callbacks=[logger])

W0829 18:36:32.558413 20452 deprecation_wrapper.py:119] From C:\Users\casey\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0829 18:36:32.603895 20452 deprecation_wrapper.py:119] From C:\Users\casey\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0829 18:36:32.611962 20452 deprecation_wrapper.py:119] From C:\Users\casey\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0829 18:36:32.768714 20452 deprecation_wrapper.py:119] From C:\Users\casey\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0829 18:36:37.004478 20452 deprecation_wrapper.py:119] From C:\Users\casey\Anac

Epoch 1/50
 - 0s - loss: 0.0169
Epoch 2/50
 - 0s - loss: 0.0032
Epoch 3/50
 - 0s - loss: 0.0015
Epoch 4/50
 - 0s - loss: 9.1098e-04
Epoch 5/50
 - 0s - loss: 5.1822e-04
Epoch 6/50
 - 0s - loss: 3.0205e-04
Epoch 7/50
 - 0s - loss: 2.0601e-04
Epoch 8/50
 - 0s - loss: 1.6240e-04
Epoch 9/50
 - 0s - loss: 1.1810e-04
Epoch 10/50
 - 0s - loss: 9.6540e-05
Epoch 11/50
 - 0s - loss: 8.9356e-05
Epoch 12/50
 - 0s - loss: 6.7844e-05
Epoch 13/50
 - 0s - loss: 5.6235e-05
Epoch 14/50
 - 0s - loss: 5.2814e-05
Epoch 15/50
 - 0s - loss: 4.6795e-05
Epoch 16/50
 - 0s - loss: 4.5136e-05
Epoch 17/50
 - 0s - loss: 4.0333e-05
Epoch 18/50
 - 0s - loss: 3.5412e-05
Epoch 19/50
 - 0s - loss: 3.2808e-05
Epoch 20/50
 - 0s - loss: 3.0273e-05
Epoch 21/50
 - 0s - loss: 3.1052e-05
Epoch 22/50
 - 0s - loss: 3.0211e-05
Epoch 23/50
 - 0s - loss: 2.9904e-05
Epoch 24/50
 - 0s - loss: 3.4065e-05
Epoch 25/50
 - 0s - loss: 3.1824e-05
Epoch 26/50
 - 0s - loss: 2.7960e-05
Epoch 27/50
 - 0s - loss: 3.0442e-05
Epoch 28/50
 - 0s - lo

In [5]:
# test results

test_data_df = pd.read_csv("sales_testing_data_scaled.csv")

X_test = test_data_df.drop('total_earnings', axis=1)
Y_test = test_data_df['total_earnings']

test_error_rate = model.evaluate(X_test, Y_test, verbose=0)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))

The mean squared error (MSE) for the test data set is: 3.361954042338766e-05


In [6]:
# to export model as a TF model, we need to use TF-specific code

# first we need to create a saved model builder object - only argument is name of folder where u want to save it

model_builder = tf.saved_model.builder.SavedModelBuilder("exported_model")

AssertionError: Export directory already exists, and isn't empty. Please choose a different export directory, or delete all the contents of the specified directory: exported_model

In [ ]:
# we need to tell TF which specific inputs and outputs we want to use when making predictions

# we pass in the input and output methods from the Keras model we built above

inputs = {'input':tf.compat.v1.saved_model.build_tensor_info(model.input)}

outputs = {'earnings':tf.compat.v1.saved_model.build_tensor_info(model.output)}

In [ ]:
# we have to create a TF signature def to tell TF how to run prediction function of the model

# this code will be the same every time

signature_def = tf.saved_model.signature_def_utils.build_signature_def(
    inputs = inputs,
    outputs = outputs,
    method_name = tf.saved_model.signature_constants.PREDICT_METHOD_NAME
)

In [ ]:
# we also add a meta graph and variables to save structure and trained weights of the model

# we pass in a reference to current keras session, then add tags so Tensorflow knows this model is for serving users

# then we pass in signature def created above and save it

model_builder.add_meta_graph_and_variables(
    K.get_session(),
    tags = [tf.saved_model.tag_constants.SERVING],
    signature_def_map = {tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:signature_def}
)

model_builder.save()

# Using Google Cloud

We can use a Gmail account to sign into the Google Cloud platform

Once we get there, we need to create a new project where we will run the model

With the new project selected at the top, we need to enable ML service under APIs & Services -> Library -> Cloud Machine Learning Engine

Note the Google will only charge per request on the model so servers are only used as needed

We then had to download the Google SDK and sign into Gmail account

There are two things we need to do to upload the model to the cloud:

1. Upload model files to Google Cloud storage bucket
2. Create new Google ML model using uploaded files

Once in the cloud, we can test the model on the sample input prescaled data from the lecture files (in JSON format)

We do this by opening regular command prompt and creating a unique storage bucket in the right folder directory with:

gsutil mb -l us-central1 gs://keras-class-052694

We then enter the following code to move the model to the storage bucket as earnings_v1:

gsutil cp -R exported_model/* gs://keras-class-052694/earnings_v1

Now that model files are on Google servers, we need to tell ML engine to create a new model:

gcloud ai-platform models create earnings_model --regions us-central1

Since we can make multiple models, we need to create the first version of the model:

gcloud ai-platform versions create v1 --model=earnings_model --origin=gs://keras-class-052694/earnings_v1 --runtime-version=1.2

Now we can predict values with this model (NOT WORKING - ERROR SHOWS UP AFTER THIS RUNS):

gcloud ai-platform predict --model=earnings --json-instances=sample_input_prescaled.json

# Using Google APIs

We can use a variety of languages to pull data from Google APIs

To use a cloud based ML model from another program, we need two things:

1. Permission to make calls to the cloud (credentials file)
2. Write code to call cloud based model

To make credentials, you need to go to console.cloud.google.com -> API Manager -> Credentials -> Create Service Account Key

Then you need to make an account name and select Project -> Viewer role

Once you click create, move the downloaded json file to the folder you are working in

Code below from lecture shows how to connect

In [7]:
from oauth2client.client import GoogleCredentials
import googleapiclient.discovery

# Change this values to match your project
PROJECT_ID = "keras-class-052694"
MODEL_NAME = "earnings_model_1"
CREDENTIALS_FILE = "credentials.json"

In [8]:
# These are the values we want a prediction for
inputs_for_prediction = [
    {"input": [0.4999, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.5]}
]

In [9]:
# Connect to the Google Cloud-ML Service

credentials = GoogleCredentials.from_stream(CREDENTIALS_FILE)
service = googleapiclient.discovery.build('ml', 'v1', credentials=credentials)

W0829 18:38:01.782690 20452 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "C:\Users\casey\Anaconda3\lib\site-packages\googleapiclient\discovery_cache\__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\casey\Anaconda3\lib\site-packages\googleapiclient\discovery_cache\file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\casey\Anaconda3\lib\site-packages\googleapiclient\discovery_cache\file_cache.py", line 37, in <module>
    from oauth2client.locked_file import Lo

In [12]:
# Connect to our Prediction Model

name = 'projects/{}/models/{}'.format(PROJECT_ID, MODEL_NAME)

response = service.projects().predict(name=name, body={'instances': inputs_for_prediction}).execute()

# Report any errors
if 'error' in response:
    raise RuntimeError(response['error'])

# Grab the results from the response object
results = response['predictions']

# Print the results!
print(results)

W0829 18:39:59.126796 20452 http.py:118] Invalid JSON content from response: b'{\n  "error": {\n    "code": 403,\n    "message": "Permission denied on resource project keras-class-052694.",\n    "status": "PERMISSION_DENIED",\n    "details": [\n      {\n        "@type": "type.googleapis.com/google.rpc.Help",\n        "links": [\n          {\n            "description": "Google developer console API key",\n            "url": "https://console.developers.google.com/project/keras-class-052694/apiui/credential"\n          }\n        ]\n      }\n    ]\n  }\n}\n'


HttpError: <HttpError 403 when requesting https://ml.googleapis.com/v1/projects/keras-class-052694/models/earnings_model_1:predict?alt=json returned "Permission denied on resource project keras-class-052694.". Details: "[{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Google developer console API key', 'url': 'https://console.developers.google.com/project/keras-class-052694/apiui/credential'}]}]">